# 学习 Qwen2.5-VL 模型

> 📚 **本 Notebook 帮助你理解和使用 Qwen2.5-VL 模型**  
> 包含模型加载、图像处理、推理和与 F-LMM 的集成

---

## 📖 目录

1. [Qwen2.5-VL 简介](## 1. Qwen2.5-VL 简介)
2. [环境准备](#环境准备)
3. [模型加载](#模型加载)
4. [图像处理](#图像处理)
5. [基础推理](#基础推理)
6. [验证：图像 Token 的扩展机制](#验证图像-token-的扩展机制) ⭐ **重要**
7. [理解 Token 编码](#理解-token-编码)
8. [提取注意力和隐藏层](#提取注意力和隐藏层)
9. [与 F-LMM 集成](#与-f-lmm-集成)
10. [常见问题](#常见问题)
11. [参考资源](#参考资源)


## 1. Qwen2.5-VL 简介

### 🎯 核心特点

- **动态分辨率**：自动适应不同图像尺寸
- **原生 ViT**：patch_size=14
- **多模态融合**：图像和文本深度融合
- **特殊 token**：`<|vision_start|>` (151652), `<|vision_end|>` (151653), `<|image_pad|>` (151655)

### 📊 与其他模型对比

| 特性 | Qwen2.5-VL | LLaVA | DeepSeek-VL |
|------|-----------|-------|-------------|
| 分辨率 | 动态 | 固定 | 固定 |
| pixel_values | 2D | 4D | 4D |
| image_grid_thw | 必需 | 不需要 | 不需要 |

### 🔑 关键概念

**image_grid_thw**: `[temporal, height_grids, width_grids]`

| 维度 | 含义 | 说明 |
|------|------|------|
| **T (temporal)** | 时间维度/帧数 | 对于**单张静态图像，T=1**；对于视频，T=视频帧数 |
| **H (height)** | 高度网格数 | 图像在高度方向上被切分成多少个 patch |
| **W (width)** | 宽度网格数 | 图像在宽度方向上被切分成多少个 patch |

**重要**: 
- Qwen2.5-VL 对单张图像 T=1（**不是 2**）
- 旧版 Qwen-VL 可能使用 T=2（复制帧），但 Qwen2.5-VL 已改进
- 图像 token 总数 = T × H × W (对于单张图像就是 H × W)


## 2. 环境准备


In [2]:
# 设置使用 3 号 GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# 你的代理地址
proxy_url = "http://10.156.216.30:16371" 

# 设置 HTTP 和 HTTPS 代理
os.environ['http_proxy'] = proxy_url
os.environ['https_proxy'] = proxy_url
os.environ['HTTP_PROXY'] = proxy_url # 建议同时设置大小写
os.environ['HTTPS_PROXY'] = proxy_url
# 检查环境
import sys
print(f"Python: {sys.version}")

try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU 数量: {torch.cuda.device_count()}")
        print(f"当前 GPU: {torch.cuda.current_device()}")
        print(f"GPU 名称: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("PyTorch not installed!")

try:
    import transformers
    print(f"Transformers: {transformers.__version__}")
except ImportError:
    print("Transformers not installed!")


Python: 3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]
PyTorch: 2.2.2+cu118
CUDA available: True
GPU 数量: 8
当前 GPU: 0
GPU 名称: NVIDIA A800 80GB PCIe
Transformers: 4.51.3


In [3]:
# 导入必要的库
import torch
import numpy as np
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
import matplotlib.pyplot as plt

print("✅ 导入成功！")


✅ 导入成功！


## 3. 模型加载

### 3.1 加载 Processor 和模型


In [4]:
# 模型名称
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"

print(f"加载 Processor: {model_name}")
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

print(f"✅ Processor 加载成功")
print(f"   类型: {type(processor)}")

加载 Processor: Qwen/Qwen2.5-VL-3B-Instruct


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Processor 加载成功
   类型: <class 'transformers.models.qwen2_5_vl.processing_qwen2_5_vl.Qwen2_5_VLProcessor'>


In [5]:
merge_size=getattr(processor, 'merge_size', None)
print(f"merge_size: {merge_size}")

merge_size: None


In [6]:
# 检查特殊 token
tokenizer = processor.tokenizer
vision_tokens = {
    '<image>': tokenizer.convert_tokens_to_ids('<|image|>'),
    '<|vision_start|>': tokenizer.convert_tokens_to_ids('<|vision_start|>'),
    '<|vision_end|>': tokenizer.convert_tokens_to_ids('<|vision_end|>'),
    '<|image_pad|>': tokenizer.convert_tokens_to_ids('<|image_pad|>')
}

print("\nVision Token IDs:")
for token, token_id in vision_tokens.items():
    print(f"  {token}: {token_id}")



Vision Token IDs:
  <image>: None
  <|vision_start|>: 151652
  <|vision_end|>: 151653
  <|image_pad|>: 151655


In [ ]:
# 方式1：官方推荐（用 dict）
messages = [{
    "role": "user", 
    "content": [
        {"type": "image"},  # 不直接写 <image>
        {"type": "text", "text": "test"}
    ]
}]
text1 = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("方式1输出：", text1)

# 方式2：直接在文本里写 <image>
text2 = "<image>test"
print("方式2输出：", text2)

In [ ]:
# 加载模型（首次加载会下载，请耐心等待）
print("加载模型...")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"✅ 模型加载成功！")
print(f"   设备: {device}")
print(f"   参数量: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B")


In [ ]:
configuration = model.config
configuration

In [ ]:
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

## 4. 图像处理和推理

### 4.1 准备测试图像


In [ ]:
# 创建测试图像（或加载真实图像）
def create_test_image(size=(640, 480)):
    return Image.new('RGB', size, color=(100, 150, 200))

# 或使用: image = Image.open('path/to/image.jpg')
image = Image.open('data/cot/flickr30k/807129.jpg')
# image = create_test_image()

print(f"图像尺寸: {image.size}")
plt.figure(figsize=(6, 4))
plt.imshow(image)
plt.title(f"测试图像 ({image.size[0]}x{image.size[1]})")
plt.axis('off')
plt.show()


### 4.2 处理图像并查看 inputs


In [ ]:
# 构建消息
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "<image>Describe this image."}
        ]
    }
]

# 处理 tokennize为True的话输出的是token
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(text)
inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True)
inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

print("处理后的输入:")
for key, value in inputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"  {key}: {value}")

# 分析 image_grid_thw
image_grid_thw = inputs['image_grid_thw']
t, h, w = image_grid_thw[0].tolist()
print(f"\n📊 image_grid_thw: [t={t}, h={h}, w={w}]")
print(f"   图像 token 数量: {h * w}")


In [ ]:
# 直接使用image_processor处理图像
image_inputs = processor.image_processor(images=image, return_tensors="pt")
print("Image processor output:")
print(f"pixel_values shape: {image_inputs['pixel_values'].shape}")
print(f"image_grid_thw: {image_inputs['image_grid_thw']}")

# 计算patch数量
patch_size = 14
height, width = image.size
scaled_h = (height + patch_size - 1) // patch_size
scaled_w = (width + patch_size - 1) // patch_size
print(f"\nManual calculation:")
print(f"Original image size: {width}x{height}")
print(f"Scaled to patches: {scaled_h}x{scaled_w} patches")
print(f"Total patches: {scaled_h * scaled_w}")

 `image_grid_thw` 是在 **Qwen-VL**（通义千问视觉语言模型）系列模型中，用于处理多模态输入时的一个**关键参数**。

这里的 `t` 代表的是：

***

### T: Temporal (时间维度/帧数)

在 `image_grid_thw` 这个四元组中，各个字母的含义如下：

| 字母 | 英文含义 | 中文含义 | 作用说明 |
| :---: | :---: | :---: | :--- |
| **T** | **Temporal** | **时间/帧数** | **图像序列的长度。** 对于单张静态图像，Qwen-VL 为了统一处理流程（使其与视频处理兼容，并利用其 3D 卷积的 Patch 提取） |
| **H** | **Height** | **高度** | 图像/帧被切分成 Patch 后的**高度方向的 Patch 数量**。 |
| **W** | **Width** | **宽度** | 图像/帧被切分成 Patch 后的**宽度方向的 Patch 数量**。 |

**简而言之，`image_grid_thw` 用于记录视觉特征在时间和空间上的网格大小，指导模型如何解释和处理输入的视觉 Tokens。**

对于 Qwen2.5-VL 这种多模态模型：

* 当输入是**单张图片**时，`T` 的值通常被设定为 **1** 
* 当输入是**多帧视频**时，`T` 就代表视频的**帧数**。

这个参数确保了视觉特征在进入模型的 Transformer 编码器之前，能够被正确地进行多模态旋转位置编码（Multi-modal RoPE）等操作。

## 5. 推理和提取注意力

### 5.1 生成文本


## 6. 验证：图像 Token 的扩展机制

### 🔬 **核心问题**：Qwen2.5-VL 如何将图像转换为 tokens？

**关键疑问**：
- F-LMM 配置文件中使用 `<image>` 占位符
- Qwen tokenizer 词表中有 `<|image_pad|>` token
- 这两者是什么关系？扩展过程是怎样的？

让我们通过 4 个实验来揭示完整流程！


In [ ]:
# ========== 实验 1: 检查 tokenizer 词表中的特殊 tokens ==========
print("=" * 80)
print("【实验 1】检查 tokenizer 词表中的特殊 tokens")
print("=" * 80)

tokenizer = processor.tokenizer

# 测试关键 tokens
test_tokens = [
    "<image>",           # 用户使用的占位符
    "<|image_pad|>",    # 实际的图像 padding token
    "<|vision_start|>",  # Vision 序列开始
    "<|vision_end|>",    # Vision 序列结束
]

print(f"\n{'Token':<20} | {'ID':>10} | {'编码后 IDs':>20} | 解码后")
print("-" * 80)

for token in test_tokens:
    try:
        # 方法1: convert_tokens_to_ids (将 token 字符串转为 ID)
        token_id = tokenizer.convert_tokens_to_ids(token)
        
        # 方法2: encode (将字符串编码为 ID 序列)
        encoded_ids = tokenizer.encode(token, add_special_tokens=False)
        
        # 解码回文本
        decoded = tokenizer.decode(encoded_ids, skip_special_tokens=False)
        
        print(f"{token:<20} | {token_id:>10} | {str(encoded_ids):>20} | '{decoded}'")
    except Exception as e:
        print(f"{token:<20} | ERROR: {e}")


In [ ]:
# ========== 实验 2: apply_chat_template 如何插入占位符 ==========
print("\n" + "=" * 80)
print("【实验 2】apply_chat_template 如何处理图像")
print("=" * 80)

# 使用 messages 格式
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe this image."}
        ]
    }
]

# 生成带占位符的文本
text_with_placeholder = processor.apply_chat_template(
    messages, 
    tokenize=False,  # 先不 tokenize，只生成文本
    add_generation_prompt=True
)

print(f"\napply_chat_template 生成的文本（tokenize=False）:")
print("-" * 80)
print(text_with_placeholder)
print("-" * 80)

# 检查包含哪些占位符
has_image = "<image>" in text_with_placeholder
has_vision_start = "<|vision_start|>" in text_with_placeholder
has_image_pad = "<|image_pad|>" in text_with_placeholder
has_vision_end = "<|vision_end|>" in text_with_placeholder

print(f"\n【发现】:")
print(f"  - 包含 '<image>': {has_image}")
print(f"  - 包含 '<|vision_start|>': {has_vision_start}")
print(f"  - 包含 '<|image_pad|>': {has_image_pad}")
print(f"  - 包含 '<|vision_end|>': {has_vision_end}")

# 统计 <|image_pad|> 数量
image_pad_count_in_text = text_with_placeholder.count("<|image_pad|>")
print(f"\n  ✓ apply_chat_template 插入了 {image_pad_count_in_text} 个 '<|image_pad|>' 占位符")
print(f"  (注意：这只是初始占位符，稍后会被扩展)")


In [ ]:
# ========== 实验 3: processor.__call__ 如何扩展占位符 ==========
print("\n" + "=" * 80)
print("【实验 3】processor.__call__ 如何将 1 个 <|image_pad|> 扩展为 N 个")
print("=" * 80)
print(f"从实验2我们知道，apply_chat_template 只插入了 {image_pad_count_in_text} 个 <|image_pad|>")
print(f"现在我们看 processor(text + images) 如何根据图像尺寸扩展它...")

# Step 1: 处理完整的文本+图像
inputs_full = processor(
    text=[text_with_placeholder],  # 包含 <image> 的文本
    images=[image],                 # 实际图像
    return_tensors="pt",
    padding=False
)

print(f"\n处理后的结果:")
print(f"  - input_ids shape: {inputs_full['input_ids'].shape}")
print(f"  - pixel_values shape: {inputs_full['pixel_values'].shape}")

if 'image_grid_thw' in inputs_full:
    T, H, W = inputs_full['image_grid_thw'][0].tolist()
    print(f"  - image_grid_thw: T={T}, H={H}, W={W}")
    expected_image_tokens = T * H * W//4
    print(f"  - 预期图像 tokens 数量: {T} × {H} × {W} = {expected_image_tokens}")

# Step 2: 统计 <|image_pad|> 的数量
image_pad_token_id = tokenizer.convert_tokens_to_ids("<|image_pad|>")
input_ids_list = inputs_full['input_ids'][0].tolist()
image_pad_count = input_ids_list.count(image_pad_token_id)

print(f"\n【关键证据】:")
print(f"  - <|image_pad|> token ID: {image_pad_token_id}")
print(f"  - input_ids 中 <|image_pad|> 的数量: {image_pad_count}")
print(f"  - 预期数量（从 image_grid_thw）: {expected_image_tokens}")

if image_pad_count == expected_image_tokens:
    print(f"\n  ✅ 完美匹配！")
    print(f"  ✅ 这证明了：processor.__call__ 将 {image_pad_count_in_text} 个 <|image_pad|> 占位符")
    print(f"     扩展为了 {image_pad_count} 个 <|image_pad|> tokens！")
    print(f"  ✅ 扩展倍数: {image_pad_count}/{image_pad_count_in_text} = {image_pad_count//image_pad_count_in_text if image_pad_count_in_text > 0 else 'N/A'}")
else:
    print(f"\n  ⚠️  差异: {abs(image_pad_count - expected_image_tokens)}")

# Step 3: 查看特殊 tokens 的位置
print(f"\n【Token 分布】:")
vision_start_id = tokenizer.convert_tokens_to_ids("<|vision_start|>")
vision_end_id = tokenizer.convert_tokens_to_ids("<|vision_end|>")

vision_start_pos = [i for i, tid in enumerate(input_ids_list) if tid == vision_start_id]
vision_end_pos = [i for i, tid in enumerate(input_ids_list) if tid == vision_end_id]
image_pad_first = input_ids_list.index(image_pad_token_id) if image_pad_count > 0 else -1
image_pad_last = len(input_ids_list) - 1 - input_ids_list[::-1].index(image_pad_token_id) if image_pad_count > 0 else -1

print(f"  - <|vision_start|> 位置: {vision_start_pos}")
print(f"  - <|image_pad|> 首次出现: 位置 {image_pad_first}")
print(f"  - <|image_pad|> 最后出现: 位置 {image_pad_last}")
print(f"  - <|vision_end|> 位置: {vision_end_pos}")
print(f"  - 总 token 数: {len(input_ids_list)}")


In [ ]:
# ========== 实验 4: 可视化完整的 Token 序列 ==========
print("\n" + "=" * 80)
print("【实验 4】可视化完整的 Token 序列")
print("=" * 80)

# 解码每个 token
print(f"\n完整 input_ids 序列 (前 50 个 tokens):")
print("-" * 80)

for i, token_id in enumerate(input_ids_list[:50]):
    # 解码单个 token
    token_text = tokenizer.decode([token_id], skip_special_tokens=False)
    
    # 标记特殊 tokens
    if token_id == image_pad_token_id:
        marker = " 👉 <|image_pad|>"
    elif token_id == vision_start_id:
        marker = " 🔵 <|vision_start|>"
    elif token_id == vision_end_id:
        marker = " 🔴 <|vision_end|>"
    else:
        marker = ""
    
    print(f"  [{i:3d}] ID={token_id:6d} → '{token_text}'{marker}")

if len(input_ids_list) > 50:
    print(f"  ... (省略剩余 {len(input_ids_list) - 50} 个 tokens)")

print("\n" + "=" * 80)
print("【结论】")
print("=" * 80)
print("""
通过以上 4 个实验，我们揭示了 Qwen2.5-VL 处理图像 token 的完整流程：

1️⃣  apply_chat_template(messages) 会检测到图像，插入初始占位符序列：
    <|vision_start|><|image_pad|><|vision_end|>（只有 1 个 <|image_pad|>）

2️⃣  processor(text, images) 会根据实际图像尺寸（image_grid_thw）：
    - 计算需要的 token 数量: N = T × H × W//4（merge_size=2）
    - 将那 1 个 <|image_pad|> 扩展为 N 个 <|image_pad|>

3️⃣  最终的 input_ids 包含：
    - 文本 tokens
    - N 个图像 tokens（都是 <|image_pad|> 的 token ID）
    - 其他特殊 tokens（<|vision_start|>, <|vision_end|> 等）

4️⃣  关键区别：
    - <image>: 这是用户在配置文件中使用的字符串（F-LMM 项目中）
    - <|image_pad|>: 这是 Qwen tokenizer 词表中真正的 token
    - apply_chat_template 直接使用 <|vision_*|> 格式，不使用 <image>

这就是 Qwen2.5-VL 处理图像 token 的完整机制！
""")


### 💡 补充说明：F-LMM 配置文件中的 `<image>` 是什么？

在 F-LMM 的配置文件中你会看到：
```python
prompt = '<image>' + "Please give me a description of the image."
image_token = '<image>'
```

**`<image>` 的真实作用**：
1. **它是一个字符串标记**，用于在 prompt 中指示"图像应该放在这里"
2. **但它不会直接出现在 `apply_chat_template` 的输出中**（如实验2所示）
3. **F-LMM 的处理流程**：
   ```python
   # 配置文件中的 prompt
   prompt = "<image>Describe this image."
   
   # F-LMM 会构造 messages 格式
   messages = [{
       "role": "user", 
       "content": [
           {"type": "image", "image": actual_image},
           {"type": "text", "text": "Describe this image."}
       ]
   }]
   
   # apply_chat_template 将其转换为
   # "<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>"
   ```

**总结**：
- `<image>` 是 F-LMM 项目的**约定字符串**，用于配置和代码中标记图像位置
- Qwen 的 `apply_chat_template` 不使用 `<image>`，而是直接生成 `<|vision_start|><|image_pad|><|vision_end|>` 序列
- F-LMM 的包装类（`QwenImageProcessorWrapper`）负责处理这个转换


In [ ]:
# 生成文本
with torch.no_grad():
    output_ids = model.generate(**inputs, max_new_tokens=50)
    
# 从模型完整的生成序列中，去掉原始输入 prompt 的部分，保留模型真正生成的新 tokens。
trimmed_ids = [
    out_ids[len(in_ids):]
    for in_ids, out_ids in zip(inputs['input_ids'], output_ids)
]

generated_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
generated_text_trimmed = processor.batch_decode(trimmed_ids, skip_special_tokens=True)[0]
print("="*50)
print("生成的文本:")
print("="*50)
print(generated_text)
print("="*50)
print("="*50)
print("生成的trimmed文本:")
print("="*50)
print(generated_text_trimmed)
print("="*50)


### 5.2 提取注意力和隐藏层（F-LMM 需要）


In [ ]:
# 提取注意力和隐藏层
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True, output_attentions=True)

print("✅ 提取完成！")
print(f"   logits: {outputs.logits.shape}")
print(f"   hidden_states: {len(outputs.hidden_states)} 层")
print(f"   attentions: {len(outputs.attentions)} 层")
print(f"   每层注意力形状: {outputs.attentions[0].shape}")

# 找到图像 token 的位置
input_ids = inputs['input_ids'][0]
vision_start_id, vision_end_id = 151652, 151653
vision_start_mask = (input_ids == vision_start_id)
vision_end_mask = (input_ids == vision_end_id)

if vision_start_mask.any() and vision_end_mask.any():
    start_pos = vision_start_mask.nonzero()[0].item()
    end_pos = vision_end_mask.nonzero()[0].item()
    print(f"\n📍 图像 token 位置: [{start_pos+1}, {end_pos})")
    print(f"   数量: {end_pos - start_pos - 1}")


## 6. 与 F-LMM 集成

### F-LMM 使用 Qwen 的关键步骤：

1. **冻结 Qwen 模型参数** - `model.requires_grad_(False)`
2. **提取 hidden_states 和 attentions** - 如上所示
3. **按 mask_ids 分组注意力** - 根据文本 token 对应的 mask
4. **重塑注意力为空间维度** - 使用 image_grid_thw
5. **通过 UNet 生成粗粒度 mask**
6. **使用 SAM 细化 mask**

### 关键代码示例（在 flmm/models/frozen_qwen.py 中）：

```python
# 提取图像 token 的注意力
images_seq_mask = torch.zeros_like(input_ids, dtype=torch.bool)
images_seq_mask[start_pos+1:end_pos] = True

attentions = []
for attn in outputs.attentions:
    attn_image = attn[:, :, images_seq_mask]
    attentions.append(attn_image)

# 重塑为空间维度
attentions = [attn.view(*attn.shape[:-1], h, w) for attn in attentions]
```


## 7. 常见问题

### Q1: 为什么需要 image_grid_thw？
**A:** Qwen2.5-VL 使用动态分辨率，需要 `image_grid_thw` 告诉模型图像被分成了多少个 patch。缺少它会导致 `TypeError`。

### Q2: pixel_values 为什么是 2D 的？
**A:** 这是 Qwen 的特殊设计。传统模型用 4D `[B, C, H, W]`，Qwen 用 2D `[num_patches, patch_dim]`。

### Q3: Token 数量不匹配的警告正常吗？
**A:** 是的！`Warning: Image token count X != Y` 是正常的，代码会自动推断正确的空间维度。

### Q4: 如何运行完整训练？
**A:** 查看文档：
```bash
cat doc/02-training/train.md
cat doc/04-qwen-adaptation/QWEN_MODEL_ADAPTATION.md
```


## 8. 参考资源

### 📚 项目文档
- **主索引**: `doc/README.md`
- **Qwen 适配指南**: `doc/04-qwen-adaptation/QWEN_MODEL_ADAPTATION.md`
- **故障排除**: `doc/05-troubleshooting/README.md`
- **测试文档**: `tests/README.md`

### 🧪 运行测试
```bash
cd tests
python test_frozen_qwen.py       # 完整测试
python diagnose_image_grid_thw.py  # 诊断工具
```

### 🔗 外部链接
- [HuggingFace Model](https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct)
- [F-LMM 论文](https://arxiv.org/abs/2406.05821)

### 💻 相关代码
- 模型实现: `flmm/models/frozen_qwen.py`
- 配置文件: `configs/qwen/frozen_qwen2_5_vl_3b_instruct_unet_sam_l_refcoco_png.py`

---

## 🎉 总结

你已经学习了：
1. ✅ Qwen2.5-VL 的核心特点和与其他模型的区别
2. ✅ 模型加载、图像处理和推理
3. ✅ 提取注意力和隐藏层（F-LMM 需要）
4. ✅ 理解 image_grid_thw 和 pixel_values
5. ✅ 与 F-LMM 的集成方式

### 下一步
- 运行完整训练: `doc/02-training/train.md`
- 深入理解架构: `doc/01-architecture/gykreadme.md`
- 查看更多示例: `scripts/demo/`

**最后更新**: 2025-11-09  
**作者**: AI Assistant


In [ ]:
# 导入必要的库
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from PIL import Image
import requests

# 加载模型和处理器
model = Qwen2_5_VLForConditionalGeneration.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# 准备图像和文本
image = Image.open(requests.get("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg", stream=True).raw)
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe the image."},
        ],
    }
]

# 使用processor处理输入
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs_tensor = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

print("处理后的输入:")
for key, value in inputs_tensor.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"  {key}: {value}")

# 生成输出
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)